In [1]:
# import warnings
# warnings.filterwarnings("ignore")

import csv
import numpy as np
import pandas as pd
import scipy

import os, os.path
import sys

# to move files from one directory to another
import shutil

In [2]:
im_dir = "/Users/hn/Documents/01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/"

In [3]:
a_dir = "/Users/hn/Documents/01_research_data/NASA/"
param_dir = "/Users/hn/Documents/01_research_data/NASA/parameters/"
evaluation_set = pd.read_csv(param_dir + "evaluation_set.csv", low_memory=False)

In [4]:
evaluation_set.head(2)

,ID,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,100010_WSDA_SF_2017,alfalfa hay,center pivot,wsda,34,34.310305,2017/09/12,Grant
1,100204_WSDA_SF_2017,alfalfa hay,center pivot,wsda,62,61.826535,2017/08/09,Grant


In [5]:
all_files = [x for x in os.listdir(im_dir) if x.endswith(".png")]

In [6]:
TOA_files = [x for x in all_files if ("TOA" in x)]
corrected_files = [x for x in all_files if ("corrected" in x)]

NDVIs = [x for x in all_files if not("corrected" in x)]
NDVIs = [x for x in NDVIs if not("TOA" in x)]

In [7]:
df = pd.DataFrame(index=range(len(NDVIs)), 
                 columns=["ID", 'NDVI_TS_Name', 'corrected_RGB', 
                          'TOA_RGB', "latitude", "longitude",
                          "Question Text", "Text box Text"])

df["Question Text"] = "How would you label this field?"
df["Text box Text"] = "Notes if you want to add any."

for counter, file in enumerate(NDVIs):
    L = file.split("_")
    L.pop(0)
    field_ID = "_".join(L)[:-4]
    # print (file, counter, field_ID, sep = ", ")
    
    # pick corrected name
    TOA_file_name = [x for x in TOA_files if field_ID in x]
    correct_file_name = [x for x in corrected_files if field_ID in x]
    
    df.loc[counter, "ID"] = field_ID
    df.loc[counter, "NDVI_TS_Name"] = file
    df.loc[counter, "corrected_RGB"] = correct_file_name[0]
    df.loc[counter, "TOA_RGB"] = TOA_file_name[0]
    
    df.loc[counter, "longitude"] = TOA_file_name[0].split("_")[5]
    df.loc[counter, "latitude"] = TOA_file_name[0].split("_")[4]
    


In [8]:
df.head(2)

,ID,NDVI_TS_Name,corrected_RGB,TOA_RGB,latitude,longitude,Question Text,Text box Text
0,46092_WSDA_SF_2018,FranklinYakima2018_46092_WSDA_SF_2018.png,46092_WSDA_SF_2018_46.38682332_-120.21093938_c...,46092_WSDA_SF_2018_46.38682332_-120.21093938_T...,46.38682332,-120.21093938,How would you label this field?,Notes if you want to add any.
1,106176_WSDA_SF_2017,Grant2017_106176_WSDA_SF_2017.png,106176_WSDA_SF_2017_47.26307371_-119.80344481_...,106176_WSDA_SF_2017_47.26307371_-119.80344481_...,47.26307371,-119.80344481,How would you label this field?,Notes if you want to add any.


In [9]:
result = pd.merge(df, evaluation_set, on="ID")
result.head(2)

,ID,NDVI_TS_Name,corrected_RGB,TOA_RGB,latitude,longitude,Question Text,Text box Text,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,46092_WSDA_SF_2018,FranklinYakima2018_46092_WSDA_SF_2018.png,46092_WSDA_SF_2018_46.38682332_-120.21093938_c...,46092_WSDA_SF_2018_46.38682332_-120.21093938_T...,46.38682332,-120.21093938,How would you label this field?,Notes if you want to add any.,alfalfa hay,sprinkler,wsda,8,8.186591,2018/10/15 00:00:00,Yakima
1,106176_WSDA_SF_2017,Grant2017_106176_WSDA_SF_2017.png,106176_WSDA_SF_2017_47.26307371_-119.80344481_...,106176_WSDA_SF_2017_47.26307371_-119.80344481_...,47.26307371,-119.80344481,How would you label this field?,Notes if you want to add any.,apple,drip/micro-sprinkler,wsda,2,1.759810,2017/05/03,Grant


In [10]:
result.sort_values(by=['CropTyp', 'county', "ID"], inplace=True)
result.reset_index(drop=True, inplace=True)
result.head(2)

,ID,NDVI_TS_Name,corrected_RGB,TOA_RGB,latitude,longitude,Question Text,Text box Text,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,104119_WSDA_SF_2016,AdamBenton2016_104119_WSDA_SF_2016.png,104119_WSDA_SF_2016_46.83273927_-118.91209151_...,104119_WSDA_SF_2016_46.83273927_-118.91209151_...,46.83273927,-118.91209151,How would you label this field?,Notes if you want to add any.,alfalfa hay,center pivot,wsda,26,26.413875,2016/08/17 00:00:00,Adams
1,51403_WSDA_SF_2016,AdamBenton2016_51403_WSDA_SF_2016.png,51403_WSDA_SF_2016_46.75090118_-118.9797641_co...,51403_WSDA_SF_2016_46.75090118_-118.9797641_TO...,46.75090118,-118.9797641,How would you label this field?,Notes if you want to add any.,alfalfa hay,center pivot,wsda,2,2.480750,2016/08/03 00:00:00,Adams


In [11]:
eshwar_limited = result[["ID", 'NDVI_TS_Name', 'corrected_RGB', 
                         'TOA_RGB', "latitude", "longitude", 
                         "Question Text", "Text box Text"]].copy()
eshwar_limited.reset_index(drop=True, inplace=True)
eshwar_limited.head(2)

,ID,NDVI_TS_Name,corrected_RGB,TOA_RGB,latitude,longitude,Question Text,Text box Text
0,104119_WSDA_SF_2016,AdamBenton2016_104119_WSDA_SF_2016.png,104119_WSDA_SF_2016_46.83273927_-118.91209151_...,104119_WSDA_SF_2016_46.83273927_-118.91209151_...,46.83273927,-118.91209151,How would you label this field?,Notes if you want to add any.
1,51403_WSDA_SF_2016,AdamBenton2016_51403_WSDA_SF_2016.png,51403_WSDA_SF_2016_46.75090118_-118.9797641_co...,51403_WSDA_SF_2016_46.75090118_-118.9797641_TO...,46.75090118,-118.9797641,How would you label this field?,Notes if you want to add any.


In [12]:
out_name = param_dir + "Eshwar_Extensive.csv"
result.index += 1
result = result.reset_index().rename({'index':'Question_overall'}, axis = 'columns')
result.to_csv(out_name, index = False)

out_name = param_dir + "Eshwar_limited.csv"
eshwar_limited.index += 1
eshwar_limited = eshwar_limited.reset_index().rename({'index':'Question_overall'}, axis = 'columns')
eshwar_limited.to_csv(out_name, index = False)

In [13]:
eshwar_limited.head(2)

,Question_overall,ID,NDVI_TS_Name,corrected_RGB,TOA_RGB,latitude,longitude,Question Text,Text box Text
0,1,104119_WSDA_SF_2016,AdamBenton2016_104119_WSDA_SF_2016.png,104119_WSDA_SF_2016_46.83273927_-118.91209151_...,104119_WSDA_SF_2016_46.83273927_-118.91209151_...,46.83273927,-118.91209151,How would you label this field?,Notes if you want to add any.
1,2,51403_WSDA_SF_2016,AdamBenton2016_51403_WSDA_SF_2016.png,51403_WSDA_SF_2016_46.75090118_-118.9797641_co...,51403_WSDA_SF_2016_46.75090118_-118.9797641_TO...,46.75090118,-118.9797641,How would you label this field?,Notes if you want to add any.


# Break into small files

In [14]:
# result = result.reset_index().rename({'index':'Question_overall'}, axis = 'columns')
# eshwar_limited = eshwar_limited.reset_index().rename({'index':'Question_overall'}, axis = 'columns')
eshwar_limited.head(2)

,Question_overall,ID,NDVI_TS_Name,corrected_RGB,TOA_RGB,latitude,longitude,Question Text,Text box Text
0,1,104119_WSDA_SF_2016,AdamBenton2016_104119_WSDA_SF_2016.png,104119_WSDA_SF_2016_46.83273927_-118.91209151_...,104119_WSDA_SF_2016_46.83273927_-118.91209151_...,46.83273927,-118.91209151,How would you label this field?,Notes if you want to add any.
1,2,51403_WSDA_SF_2016,AdamBenton2016_51403_WSDA_SF_2016.png,51403_WSDA_SF_2016_46.75090118_-118.9797641_co...,51403_WSDA_SF_2016_46.75090118_-118.9797641_TO...,46.75090118,-118.9797641,How would you label this field?,Notes if you want to add any.


In [15]:
no_questions = 60
if eshwar_limited.shape[0] % no_questions != 0:
    no_dfs = eshwar_limited.shape[0] // no_questions + 1
else:
    no_dfs = eshwar_limited.shape[0] // no_questions

In [16]:
broken_extended_dir = param_dir + "Eshwar_extended_broken/"
broken_limited_dir = param_dir + "Eshwar_limited_broken/"

os.makedirs(broken_extended_dir, exist_ok=True)
os.makedirs(broken_limited_dir, exist_ok=True)

writer_limited = pd.ExcelWriter(param_dir + 'all_limiteds.xlsx', engine='xlsxwriter')
writer_extended = pd.ExcelWriter(param_dir + 'all_extended.xlsx', engine='xlsxwriter')


for ii in range(no_dfs):
    curr_eshwar_limited = eshwar_limited.loc[(ii*no_questions): ((ii+1) * no_questions) - 1, ]
    curr_result = result.loc[(ii*no_questions): ((ii+1) * no_questions) - 1, ]
    
    curr_eshwar_limited.reset_index(drop=True, inplace=True)
    curr_result.reset_index(drop=True, inplace=True)
    
    curr_eshwar_limited.index += 1
    curr_result.index += 1
    
    curr_eshwar_limited = curr_eshwar_limited.reset_index().rename({'index':'Question_in_set'}, axis = 'columns')
    curr_result = curr_result.reset_index().rename({'index':'Question_in_set'}, axis = 'columns')
    
    out_name = broken_limited_dir + "question_set_limited_" + str(ii+1) + ".csv"
    curr_eshwar_limited.to_csv(out_name, index = False)
    
    out_name = broken_extended_dir + "question_set_extended_" + str(ii+1) + ".csv"
    curr_result.to_csv(out_name, index = False)
    
    
    curr_eshwar_limited.to_excel(writer_limited, sheet_name= "limited_" + str(ii+1), index=False)
    curr_result.to_excel(writer_extended, sheet_name= "extended_" + str(ii+1), index=False)

writer_limited.save()
writer_extended.save()